### Exercício 1 - Extração, transformação e exploração de dados da loteria Loto Fácil
##### Tarefas:
* extrair dados do site da Caixa
* limpar os dados (vem uma lista de texto bagunçada)
    * ajeitar o df
    * ajeitar as colunas
    * apagar os nulos
* refatorar a exploração de prof Rhuan
* identificar o número mais e menos apostado
* identificar a combinação mais apostada separando os números em par, ímpar e primo

In [1]:
import pandas as pd
import requests
import sys
import collections

In [2]:
url = 'https://servicebus2.caixa.gov.br/portaldeloterias/api/resultados?modalidade=Lotofácil'
#url = sys.argv[1]
r = requests.get(url, verify=False)

/home/mirian/anaconda3/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'servicebus2.caixa.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [3]:
# replaces = {'\\r\\n': '', '"\r\n':'', '{\r\n "html": "': ''}
# r_text = r.text.replace(replaces, regex=True)
# TypeError: str.replace() takes no keyword arguments
# assim só funciona pra df.replace, str.replace requer gambiarras
r_text = r.text.replace('\\r\\n', '').replace('"\r\n','').replace('{\r\n "html": "', '')

In [4]:
list_text = pd.read_html(r_text)

In [5]:
df = list_text[0]

In [6]:
cols = df.columns
new_cols = list(col.replace('\\r\\n','') for col in cols)
df.columns = new_cols

In [7]:
df.head()

,Concurso,Data_Sorteio,Bola1,Bola2,Bola3,Bola4,Bola5,Bola6,Bola7,Bola8,...,Ganhadores_12_Números,Ganhadores_11_Números,Valor_Rateio_15_Números,Valor_Rateio_14_Números,Valor_Rateio_13_Números,Valor_Rateio_12_Números,Valor_Rateio_11_Números,Acumulado_15_Números,Estimativa_Prêmio,Valor_Acumulado_Especial
0,1,29/09/2003,2.0,3.0,5.0,6.0,9.0,10.0,11.0,13.0,...,48807.0,257593.0,"R$49.765,82","R$689,84","R$10,00","R$4,00","R$2,00","R$0,00","R$0,00","R$0,00"
1,NaN,BA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,PR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,SP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,06/10/2003,1.0,4.0,5.0,6.0,7.0,9.0,11.0,12.0,...,81252.0,478188.0,"R$596.323,70","R$1.388,95","R$10,00","R$4,00","R$2,00","R$0,00","R$0,00","R$0,00"


In [8]:
df = df[df['Bola1'] == df['Bola1']]
# um valor nulo sempre é False quando comparado
# dropna não excluiu corretamente, engoliu linhas

In [9]:
df.head()

,Concurso,Data_Sorteio,Bola1,Bola2,Bola3,Bola4,Bola5,Bola6,Bola7,Bola8,...,Ganhadores_12_Números,Ganhadores_11_Números,Valor_Rateio_15_Números,Valor_Rateio_14_Números,Valor_Rateio_13_Números,Valor_Rateio_12_Números,Valor_Rateio_11_Números,Acumulado_15_Números,Estimativa_Prêmio,Valor_Acumulado_Especial
0,1,29/09/2003,2.0,3.0,5.0,6.0,9.0,10.0,11.0,13.0,...,48807.0,257593.0,"R$49.765,82","R$689,84","R$10,00","R$4,00","R$2,00","R$0,00","R$0,00","R$0,00"
4,2,06/10/2003,1.0,4.0,5.0,6.0,7.0,9.0,11.0,12.0,...,81252.0,478188.0,"R$596.323,70","R$1.388,95","R$10,00","R$4,00","R$2,00","R$0,00","R$0,00","R$0,00"
6,3,13/10/2003,1.0,4.0,6.0,7.0,8.0,9.0,10.0,11.0,...,96244.0,608211.0,"R$400.623,70","R$2.173,36","R$10,00","R$4,00","R$2,00","R$0,00","R$0,00","R$0,00"
8,4,20/10/2003,1.0,2.0,4.0,5.0,8.0,10.0,12.0,13.0,...,123912.0,706657.0,"R$902.226,02","R$1.498,72","R$10,00","R$4,00","R$2,00","R$0,00","R$0,00","R$0,00"
10,5,27/10/2003,1.0,2.0,4.0,8.0,9.0,11.0,12.0,13.0,...,195636.0,860992.0,"R$380.017,55","R$687,49","R$10,00","R$4,00","R$2,00","R$0,00","R$0,00","R$0,00"


In [10]:
df_bolas = df[['Bola1', 'Bola2', 'Bola3', 'Bola4', 'Bola5',
              'Bola6', 'Bola7', 'Bola8', 'Bola9', 'Bola10', 'Bola11', 'Bola12',
              'Bola13', 'Bola14', 'Bola15']]

In [11]:
# número mais frequente
todas_bolas = df_bolas.stack().reset_index().rename(columns={0:'bolas'})

In [12]:
bolass = todas_bolas.astype({'bolas': 'int32'})

In [13]:
def is_prime(n):
  for i in range(2,n):
    if (n%i) == 0:
      return False
  return True

In [14]:
v_pares_2 = []
v_impares_2 = []
v_primos_2 = []
for num in bolass.bolas:
    if num > 1:
        if num % 2 == 0:
            v_pares_2.append(num)
        if num % 2 != 0:
            v_impares_2.append(num)
        if is_prime(num) == True:
            v_primos_2.append(num)

In [15]:
comb_v2 = []
for p2,i2,np2 in zip(v_pares_2,v_impares_2,v_primos_2):
       comb_v2.append(str(p2) + 'p-' + str(i2) + 'i-'+str(np2)+'np')

In [16]:
counter_v2 = collections.Counter(comb_v2)
resultado_v2 = pd.DataFrame(counter_v2.items(), columns=['Combinacao', 'Frequencia'])
resultado_v2['p_freq'] = resultado_v2['Frequencia']/resultado_v2['Frequencia'].sum()
resultado_v2 = resultado_v2.sort_values(by='p_freq')

In [17]:
print('''
A combinação de Pares, Ímpares e Primos mais frequente é: {} com a frequencia de: {}%
'''.format(resultado_v2['Combinacao'].values[-1], int((resultado_v2['p_freq'].values[-1]*100)*100)/100)
)


A combinação de Pares, Ímpares e Primos mais frequente é: 10p-9i-13np com a frequencia de: 0.17%



In [18]:
comb = resultado_v2['Combinacao'].values[-1]

In [19]:
freq = int((resultado_v2['p_freq'].values[-1]*100)*100)/100

In [20]:
print(f'A combinação de Pares, Ímpares e Primos mais frequente é: {comb} com a frequencia de: {freq}%')

A combinação de Pares, Ímpares e Primos mais frequente é: 10p-9i-13np com a frequencia de: 0.17%


In [21]:
n_mais_freq = todas_bolas.bolas.mode()
print(f'O número mais frequente é {int(n_mais_freq)}')

O número mais frequente é 20


In [22]:
# número menos frequente
n_menos_freq = todas_bolas.bolas.value_counts().index[-1]
print(f'O número menos frequente é {int(n_menos_freq)}')

O número menos frequente é 8
